### First, get the data.

For this tutorial we'll use available data from the study "Transcriptome Dynamics of the Stomatal Lineage: Birth, Amplification, and Termination of a SelfRenewing Population" by the Bergman group [link to the study] (https://www.ncbi.nlm.nih.gov/pubmed/25850675). 

Data is under the project PRJNA253731 (SRA: SRP043607) and can be found in [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE58856).

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE58856



** Note **
The experiment also contains microarray samples (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE58856). Be sure to get the SRA samples which are the RNAseq files (GSE58857).


* The matrix contains information about the experiment: ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE58nnn/GSE58856/matrix/
* Some supplementary files:L
    * ftp://ftp.ncbi.nlm.nih.gov/pub/geo/DATA/supplementary/series/GSE58856/GSE58856_RNASeq_Adrian_et_al_2014.txt.gz 

** Experiment Description **

Run in R:

```
library("GEOquery")

setwd("~/Desktop/RNASeq_Pipeline/")
dir.create("RawData",showWarnings = F)
##
gse <- getGEO("GSE58857", GSEMatrix = TRUE,destdir = "RawData/")
show(gse)
names(gse$GSE58856_series_matrix.txt.gz)
meta <- as(gse$GSE58856_series_matrix.txt.gz,"data.frame")
write.csv(file = "meta/metaData_GSE58857.csv",x = meta,quote = F,row.names = T)
```

The resulting table contains information about the experiment.

The experiment consists of protplasts separated by FACS. They range from epidermis cells, to mother and mature guard cells. To make this analysis faster we'll focus in epidermis and mature guard cells.

If we take a look at the table:

``meta[,c("source_name_ch1","title","relation"),drop=F]``




### Download Data


** To download SRA data set use the SRA toolkit**

https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=toolkit_doc&f=std

(Download the compiled binary and add it to your path.)

Example: https://www.biostars.org/p/111040/


These are files of ~1GB each so it may take some time to download them.


|SRR| Sample | Source |
|---|---| --- |
|``SRR1463325``| RNASeq - ML1p::YFP-RCI2A (ML1Y) - replicate 1 | 	Epidermis cells |
|``SRR1463326``| RNASeq - ML1p::YFP-RCI2A (ML1Y) - replicate 2 |	Epidermis cells |
|``SRR1463334``| RNASeq - E1728::GFP (E1728G) - replicate 1  | Mature guard cells|
|``SRR1463335``| RNASeq - E1728::GFP (E1728G) - replicate 2 | Mature guard cells|

Let's use wget to download these files:

Create a folder to store the files (``mkdir OriginalFiles``) and move into that folder.

>``wget ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByExp/sra/SRX/SRX627/SRX627394/SRR1463325/SRR1463325.sra``

> ``wget ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByExp/sra/SRX/SRX627/SRX627395/SRR1463326/SRR1463326.sra``

> ``wget ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByStudy/sra/SRP/SRP043/SRP043607/SRR1463334/SRR1463334.sra``

> ``wget ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByStudy/sra/SRP/SRP043/SRP043607/SRR1463335/SRR1463335.sra``


And convert them to fastq files:

>``for each in *.sra; do echo $each; fastq-dump --gzip $each .; done``

### Good Practices on RNASeq data.

Treat your data preciously. It takes a lot of effort and money to produce, so any accidental corruption of the files might result in a severe stepback.

One thing I learned when analyzing RNA seq data (and in principle one should do this with any type of raw data) was to protect and use soft symbolic links (think of it like direct access) to access it or modify the filenames. Personally I never modify the names of the raw files. If I need to do it I usually do it on the symbolic links or change the name on downstream files during the analysis using a metadata file.

Inside the OriginalFiles folder change the writing permissions for the files"

> `cd OriginalFiles`
> `chmod -w *`
> `cd ..`


Then create a new folder and create symbolic links to the original files.

> `mkdir RawData`
> `cd RawData`
> `ln -s ../OriginalFiles/* . #Create a soft link`

Check the files, they're still readable but they point to another location. If we accidentally modify or delte these files the original files won't be affected

> `ll -h`
> `gunzip -dc SRR1463325.fastq.gz | head`
> `cd .. # Exit to the main folder.`

<hr>

## 1. Quality Control on Raw Samples

<p>Let's take a look at the raw samples. It's important to check the quality of the sequences before even start the analysis. We can get an idea if something went wrong during the sequencing process, how much adapter contamination we have and if barcodes have to be trimmed.</p>

To make this tutorial faster let's just run one sample at a time. 

#### Before we begin.

Normally data like this should be run on a server, but for the purposes of this tutorial we'll look at a subset of the reads. Let's use the first 200,000 reads:

> `gunzip -dc RawData/SRR1463325.fastq.gz | head -n 800000 > RawData/testSeq.fastq` <br>
> `gzip RawData/testSeq.fastq`

<center>__Q: I set the lines to 800,000 even though we only want 200,000 reads. Do you know why?__</center>


### Run fastqc

Get acquainted with what fastqc does and what each module means: http://www.bioinformatics.babraham.ac.uk/projects/fastqc/




> `mkdir -p 00QC/Raw` <br>
> `fastqc RawData/testSeq.fastq.gz -o 00QC/Raw/`

Open the html generated inside the folder and take a look at the report. 

Since we're using only the first 200,000 sequences (out of 36,145,497) the report won't look too bad. 
If you check the _kmer content_ tab you'll see that the first ~12 bp have an increased overrepresentation of k-mers. This might be due to barcodes or adapters during the RNAseq. We need to trim these before we align the reads to the genome.


<hr>

If you want to run all samples (not recommended on your desktop), do:

> ``fastqc RawData/*.fastq.gz --noextract -o 00QC/Raw``

<hr>

<hr>
## 2. Remove Adapters/Clean Reads


** Adapter Contamination **

It is necessary to remove in short reads (~50bp) but not quite in longer reads (>100bp). 


Test known adapters with by with `grep`.

ie: <br>
``gunzip -dc SRR1463326.fastq.gz | grep "ATCTCGTATGCCGTCTTCTGCTTG"``
                                   
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAAT**ATCTCGTATGCCGTCTTCTGCTTG**
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAAT**ATCTCGTATGCCGTCTTCTGCTTG**
AATAACGTAACGTAATATCATC**ATCTCGTATGCCGTCTTCTGCTTG**AAAAAAAAAAATA
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGATCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGGCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACCCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCAGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG
AAAAATAACGTAACGTAATATCATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAA
GGAAGAGCACACGTCTGAACTCCAGTCACGCCAATATCTCGTATGCCGTCTTCTGCTTG


The sequence corresponds to the v1.5 Small RNA 3' Adapter from the Illumina. Other potential contaminating sequences * could be:

* GATCGGAAGAGCACACGTCTGAACTCCAGTCAC


See: https://support.illumina.com/content/dam/illumina-support/documents/documentation/chemistry_documentation/experiment-design/illumina-adapter-sequences_1000000002694-01.pdf.

--- <br>

__ minion and the Kraken suite__ 
If the adapter sequence is not known, the program `minion` is part of the [Kraken suite](https://www.ebi.ac.uk/research/enright/software/kraken) which are tools developed for the preprocessing of sequencing reads (adapter inference/trimming, minimizing redundancy).

`minion` is designed to infer the presence of an adapter reading the first 2 million reads. If you want to change the number of reads use the paramter `-do <NUMBER>` to change this:

* To infer the adapter:

>``minion search-adapter -i SRR1463326.fastq #-do 200000``

* To test a sequence

> ``minion search-adapter -i SRR1463326.fastq -adapter ATCTCGTATGCCGTCTTCTGCTTG #-do 200000``


<br>

** Removing barcodes **

If there are still barcodes present in the data ``fastx_trimmer`` from the ``fastx`` toolkit can be used.


For example:

``fastx_trimmer -v -f 9 -Q33`` to remove the first 8bp.


** Code used **

First, input the adapter and tabu sequences:

   
> ``tabu="GATCGGAAGAGCACACGTCTGAACTCCAGTCAC"`` <br>
> ``adapter="ATCTCGTATGCCGTCTTCTGCTTG"``

If we look at the K-mer content in the QC analysis we can see a high content of kmers in the first 10-12 nt. Let's go ahead and clip these parts of the read. We don't want to lose a lot of information (the longer the read, the better the mapping) but since these low complexity sequences are at the beggining of the read they could harm downstream analysis. Let's clip the first 11 nt. Set with `-f` set to 12 (cut the first 11 nt) for  `fastx_trimmer`. 


Create a folder to store the trimmed sequences:
>``mkdir 01_trimmed`` <br>

Run:
> ``for n in RawData/*test*.fastq.gz; do tmp=`basename $n`; tmp=${tmp%.fastq.gz}; echo Doing $tmp; gunzip -dc $n | fastx_trimmer -v -f 12 -Q33 - | reaper -geom no-bc -tabu $tabu -3pa $seqAdapt -noqc -dust-suffix 6/ACTG -dust-suffix-late 6/ACTG -nnn-check 1/1 -qqq-check 35/10 -clean-length 30 -tri 10 -polya 5 -basename 01_trimmed/$tmp ;done``



For more info on the parameters see [reaper](http://wwwdev.ebi.ac.uk/enright-dev/kraken/reaper/src/reaper-latest/doc/reaper.html) and [fastx](http://hannonlab.cshl.edu/fastx_toolkit/)



<hr>

If you want to run all samples (not recommended on your desktop), do:

> ``for n in RawData/*fastq.gz; do tmp=`basename $n`; tmp=${tmp%.fastq.gz}; echo Doing $tmp; gunzip -dc $n | fastx_trimmer -v -f 12 -Q33 - | reaper -geom no-bc -tabu $tabu -3pa $seqAdapt --noqc -dust-suffix 6/ACTG -dust-suffix-late 6/ACTG -nnn-check 1/1 -qqq-check 35/10 -clean-length 30 -tri 10 -polya 5 -basename 01_trimmed/$tmp ;done``


<hr>


<hr>
## 3. Quality Control on Clean Reads


Let's take a look on the trimmed reads and compare with the raw data.

Create a folder to store the files:
> ``mkdir 00QC/trimmed`` <br>

Run fastqc:
> ``fastqc 01_trimmed/*test*.lane.clean.gz -o 00QC/trimmed``


At least on our test data the trimming of the first 8bp seemed to have worked. The overrepresented k-mers are now gone and everything seems ok. The tab _Sequence Length Distribution_ shows a warning but this is because after trimming by quality and complexity we expect that not all reads are going to have the same length.




<br>
<br>

<hr>

If you want to run all samples (not recommended on your desktop), do:

> ``fastqc 01_trimmed/*.fastq.gz --noextract -o 00QC/trimmed``

<hr>

<hr>
### About quality control

Lots of stuff can go wrong during sequencing; at any step from the preparation of the libraries to the interpretation of results. Even before that, if the design of the experiment isn't great, results won't be great. 

Thus, some things are easier to control than others; when analyzing the sequenced libraries it is a good practice to check the quality control of both raw and processed reads before delving into the mapping and subsequent steps.

Here are some links that discuss some of these matters.  


https://sequencing.qcfail.com/articles/position-specific-failures-of-flowcells/



> # *Don't panic*

<hr>


<hr>
## 4. Alignment to a reference

After we are sure our sequences have been cleaned, we can proceed to map them back to the a reference genome or transcriptome.

### Get the data

In this example we're using Arabidopsis data, so we need to download the fasta files corresponding to the chromosomes. If the fasta of your genome is splitted in different files you'll need to concatenate them into a single file.

The _fasta.fa_ file for the index needs to have a format like:

\> Chr 1 <br>
ACTGACTGACTGACTGACTGACTGACTGACTGACTG<br>
\> Chr 2<br>
ACTGACTGACTGACTGACTGACTGACTGACTGACTG<br>
\> Chr N<br>
ACTGACTGACTGACTGACTGACTGACTGACTGACTG<br>



Luckyly there's a resource for that: go to the TAIR ftp site and get the data:

> ftp://ftp.arabidopsis.org/home/tair/Genes/TAIR10_genome_release/TAIR10_chromosome_files/TAIR10_chr_all.fas


Also, we'll need a GFF file for the annotation (to assign counts to each gene). Go ahead and download the file form the TAIR's ftp site:

> ftp://ftp.arabidopsis.org/home/tair/Genes/TAIR10_genome_release/TAIR10_gff3

<hr>

## Bowtie

There are many mapping algorithms that we can use, depending on our type of data, some tools might be better than others. Also, depending on the computational power we have available, we might prefer some program over other.


[Bowtie](http://bowtie-bio.sourceforge.net) and [Bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml) are widely use tools for hight-troughput sequencing mapping. They're designed to be fast, and can perform well on a desktop computer. Here we're dealing with short reads (\<50bp); Bowtie1 (aka just Bowtie) peforms well with these type of reads. 

They're based on the __burrow wheeler transform__ algorithm used for compression. Check the wikipedia entry to know more: https://en.wikipedia.org/wiki/Burrows–Wheeler_transform.


<hr>

### Create the Index

After downloading and concatenating the chromosome into a single file, create the index 

> `mkdir -p athGenome/bwt1_genome #use a subfolder in case you want to use the transcriptome instead of the genome`<br>
> `bowtie-build meta/genome_ath_TAIRv10.fa athGenome/bwt1_genome/athIndex #athIndex will be the prefix for the files`<br>

<hr>

### Align!


Check the bowtie manual to understand how it works and what are the parameters doing.


> `mkdir -p AlignedReads/bwt1_genome #use a subfolder in case you want to use the transcriptome instead of the genome`<br>
> ``for n in 01_trimmed/*test*.lane.clean.gz; do tmp=`basename $n`; tmp=${tmp%.lane.clean.gz}; echo Doing $tmp; gunzip -dc $n | bowtie -a --best --strata -n 1 -m 1 -p 4 --sam --tryhard athGenome/bwt1_genome/athIndex - -S AlignedReads/bwt1_genome/$tmp.sam ;done``


The output gives us information on the mapping efficiency of our reads:


\# reads processed: 83003 <br>
\# reads with at least one reported alignment: 52950 (63.79%)<br>
\# reads that failed to align: 10537 (12.69%)<br>
\# reads with alignments suppressed due to -m: 19516 (23.51%)<br>
Reported 52950 alignments to 1 output stream(s)<br>


During the trimming/cleaning process we lost some of our reads (due to low quality, low complexity and other factors):
__116997__ (reads that were thrown away, check the lint file in the trimmed folder) + __83003__ (usable, clean reads) = __200,000__ starting reads.

However we have a pretty good alignment percentage (~64% of the reads that are left have a unique position in the genome).

### Filter SAM file and convert to BAM.

The SAM format contains the information of the genomic position of each of the reads. If we check the file we'll see that all the reads (83003) are present, regardless if they were uniquely mapped or not. We need to get rid of those that we don't want.

> `grep -v "@" AlignedReads/bwt1_genome/testSeq.sam | wc -l` <br>
> 83003


Samtools allows us to filter these and to compress to another format (BAM), which compresses the information so files aren't huge.


> ``for n in AlignedReads/bwt1_genome/*test*.sam; do tmp=`basename $n`; tmp=${tmp%.sam}; echo Doing $tmp; samtools view -q 30 -b -S $n > AlignedReads/bwt1_genome/$tmp.bam ;done``

To check that the file removed the unwanted reads:

> `samtools view AlignedReads/bwt1_genome/testSeq.bam | wc -l`
> 52950 # This should be the same number in the uniquely aligned reads from the bowtie output.


The compression from SAM to BAM helps us keep smaller files. In this example, the size goes from 15Mb (SAM) to only 2Mb (BAM). Imagine how big the files would be if we were working with the original full data.

We can now go ahead and delete the .sam files to free disk space. 


<br>
<hr>

If you want to run all samples (not recommended on your desktop), do:

> ``for n in AlignedReads/bwt1_genome/*.sam; do tmp=`basename $n`; tmp=${tmp%.sam}; echo Doing $tmp; samtools view -q 30 -b -S > AlignedReads/bwt1_genome/$tmp.bam ;done``

<hr>

<hr>
## 5. Count me in.

Great! So far we've protected our files against accidental edition or deletion, checked the quality before and after trimming, and mapped to the genome.

A good fraction (~64%) of the reads are mapping to the genome. Now we want to know how many counts can we assign to each gene. 

As in the previous steps, there are different tools we can use. If you're more of an R person, the  [GenomicAlignments](http://www.bioconductor.org/packages/release/bioc/html/GenomicAlignments.html) package from bioconductor might interest you.

For this example we'll use [HTSeq](www-huber.embl.de/users/anders/HTSeq/doc/count.html), a python-based tool.

Regardless of the tool you use they have the same basic principle: having the genomic position of each read we want to compare to a reference of the annotated genes. We assign a read (count) to a gene if the overlap between them is significant.

The annotation reference will be the GFF file we previously downloaded form TAIR.


> mkdir -p Counts/bwt1_genome <br>

> ``for n in AlignedReads/bwt1_genome/*test*.bam; do tmp=`basename $n`; tmp=${tmp%.bam}; echo Doing $tmp; samtools view -h $n | samtools sort -O sam | htseq-count -f sam --stranded yes --mode "union" --idattr ID --type gene - meta/TAIR10_withTransposons.gff > Counts/bwt1_genome/$tmp.txt;done
Doing testSeq.bam``


Check it worked:

> ``head Counts/bwt1_genome/testSeq.txt`` <br>


HTSeq gives a summary of the annotation at the end of the file, counting how many of the aligned reads didn't ovlerap with any of the genes in the GFF file or how many could've been assigned to more than one gene (ambiguous). These are not considered.  

> ``tail Counts/bwt1_genome/testSeq.txt`` <br>

__no_feature	27537
__ambiguous	292
__too_low_aQual	0
__not_aligned	0
__alignment_not_unique	0

We know that we have originally 52,950 uniquely mapped reads. And we have 27537+292 sequences that weren't counted as part of any gene. Then the number of effectively counted reads is:
52950-(27537+292) = 25,121

We can prove it by running the following code:
> ``grep -v "__" Counts/bwt1_genome/testSeq.txt | cut -f2 | paste -sd+ - | bc`` <br>
> 25121

#### Count Matrix.
Using R we can pull together each of the count files to create a matrix of counts for differential expression analysis.


<br>
<hr>
If you want to run all samples (not recommended on your desktop), do:

``for n in AlignedReads/bwt1_genome/*test*.bam; do tmp=`basename $n`; tmp=${tmp%.bam}; echo Doing $tmp; samtools view -h $n | samtools sort -O sam | htseq-count -f sam --stranded yes --mode "union" --idattr ID --type gene - meta/TAIR10_withTransposons.gff > Counts/bwt1_genome/$tmp.txt;done
Doing testSeq.bam``
<hr>


<hr>
## 5. Differential Gene Expression.

Finally, we can 



#### Metadata file

Personally, I like to use a file (.txt,.csv,.tsv format) where information about the experiment is stored. The more information, the better! 

For example if you know that the experiments were run in different days that information can go into the model and account for batch or other effects that could have an influence in the data.

An example of how a good meta data looks:

|File| Sample | Source | rep |
|---|---| --- | --- |
|SRR1463325| ML1pYFP-RCI2A (ML1Y)_replicate_1 | 	Epidermis_cells | r1 |
|SRR1463326| ML1pYFP-RCI2A (ML1Y)_replicate_2 |	Epidermis_cells | r2 |
|SRR1463334| E1728GFP_(E1728G)_replicate_1  | Mature_guard_cells| r1|
|SRR1463335| E1728GFP_(E1728G)_replicate_2 | Mature_guard_cells| r2 |



The following is an example of R code for differential expression using limma/voom.

The raw counts and associated metadata files are in the github repository.

<pre>
> library(edgeR)
library(gplots)
library(calibrate)
library(RColorBrewer)
library(limma)

> GeneCounts <- read.csv("RawCounts_bwt1_genome.csv",row.names = 1) #1 gene

> meta <- read.csv("meta/meta.csv",as.is = T,row.names = 1)


> meta$SampleNameFull <-paste0(meta$Name,meta$Rep)
colnames(GeneCounts) <- meta$SampleNameFull #Assign sample names to counts table
cat(ncol(GeneCounts),"samples in the data frame \n")
ncol(GeneCounts) == nrow(meta)
dim(GeneCounts)


> cat("Removing genes with 0 counts on all conditions \n")
cat("Initial number of genes:",nrow(GeneCounts),"\n")
rmIDX <- which(rowSums(GeneCounts) == 0)
if (length(rmIDX) != 0){
  cat("Removing",length(rmIDX),"genes \n")
  GeneCounts <- GeneCounts[-rmIDX,]
  cat("Remaining number of genes:",nrow(GeneCounts),"\n")
} else {
  cat("No genes with 0 counts on all  \n")
}



> design <- model.matrix(~0+Name, data =meta)#+Lane
colnames(design) <- gsub("Name|:|-|/|Group","",colnames(design))
head(design)


> dge <- DGEList(counts=GeneCounts,remove.zeros = T) 
range(table(meta$Name))
sampleMin <- min(table(meta$Name))
minCPM <- 1
isexpr <- rowSums(cpm(dge) > minCPM) >= sampleMin #Make sure to use the minimum number of reps
table(isexpr)
cat("Removing lowly expressed genes ( <",minCPM,"cpm on at least",sampleMin,"samples) \n")
#Remove lowly expressed genes
cat("Removing",table(isexpr)[1],"genes \n")
cat("Remaining number of genes:",table(isexpr)[2],"\n")
dge <- dge[isexpr,,keep.lib.size = FALSE]


<hr>

> #Optional, do TMM along with voom
dge <- calcNormFactors(dge)
v <- voomWithQualityWeights(dge, design=design, normalization="quantile", plot=TRUE)


<hr>

> ##Check samples after normalization
library(RColorBrewer)

> ##-- Produce graphics
Targets <- meta
col.Source <- c("steelblue4", "skyblue")
> ##Boxplot
boxplot(v$E, range=0,col=col.Source[as.factor(meta$Name)], 
        ylab="log2[counts]", xlab="sample", main="voom normalized counts",
        cex.axis=0.5,las=2)


> ##MDS plots with glimma
library(Glimma)
glMDSPlot(v, labels=meta$SampleNameFull, groups=Targets[,c("Name")],
          folder=paste0(outDir,"/glimma"),
          launch=T)

<hr>

> ##Fit
v2 <- lmFit(v, design)
###do contrasts
contrastMatrix <- makeContrasts(
   "EPIvMGC"=(EPI-MGC),
  levels = design)

> fit2 <- contrasts.fit(v2, contrastMatrix)
fit2 <- eBayes(fit2)

<hr>

> ##Venn diagrams
results <- decideTests(fit2)#,lfc = logCut,p.value = pValCut)
summary(results)
vennDiagram(results,include = c("up","down"))


> ##Get normalized expression
voomNormalizedExpression <- v$E


<hr>
> ##If gene alias information is available:
geneAlias <- read.delim("meta/gene_aliases.txt",header = T, fill = NA,stringsAsFactors = F)
geneAlias <- split(geneAlias,geneAlias$locus_name)
AGI2Symbol <- lapply(geneAlias, function(x){
  idx <- grepl("^(?!A[Tt])",x[,"symbol"],perl = T); #Negates genes starting with At or AT
  if (all(idx == F)){
    paste(x[,"symbol"],collapse = "/")
  } else {
    symbols <- paste(x[idx,"symbol"],collapse = "/") }
})
AGI2Symbol <- unlist(AGI2Symbol)
head(AGI2Symbol)

<hr>

> ##-- Do DGE
logCut <- 1
pValCut <- 0.05
uniqContrasts <- colnames(contrastMatrix)

> cat("Genotypes that will be analyzed:" ,paste0(uniqContrasts),"\n")

> ###Prepare lists
DEList <- list()
DESignificant <- list()

> for (contrast in uniqContrasts){
  cat(" - - - \n")
  dropContrasts <- contrast
  contrastNames <- contrast#colnames(fit2$coefficients)[dropContrasts]
  cat("Contrast:", paste0(contrastNames),"\n")
  tmp <- topTable(fit2, coef=dropContrasts,number = Inf,sort.by = "none")
  tmp[,"Symbol"] <- rownames(tmp)
  
  > #-- Add gene symbols
  Genes <- rownames(tmp)
  idx <- intersect(names(AGI2Symbol),Genes)
  tmp[idx,"Symbol"] <- AGI2Symbol[idx]
    
  >###Change names of columns
  #Get indices of columns that correspond to logFC
  #tmpIDX <- grep(paste(contrastNames,collapse="|"),colnames(tmp))
  colnames(tmp) <- paste(contrast,colnames(tmp),sep = ".")
  DEList[[contrast]] <- tmp
  
 >##Filter
  tmpIDX <- grep("adj.P.Val",colnames(tmp))
  tmpSign <- tmp[tmp[,tmpIDX] < pValCut,] 
  nrow(tmpSign)
  DESignificant[[contrast]] <- tmpSign

  > cat ("Number of DEGs on",contrast,":",nrow(tmpSign),"\n")
  }  


> sapply(DEList,function(x){table(x[,grep("adj.P.Val",colnames(x))] < 0.05)})
> significantDE <- lapply(DEList,function(x){ x[x[,grep("adj.P.Val",colnames(x))] < 0.05,] })
> sapply(significantDE,nrow)
<hr>


> ##Get the DE genes.
DE <- significantDE[[1]]
head(DE)


</pre>


<hr>
### Resources

* About duplication in RNAseq 
    * http://proteo.me.uk/2013/09/a-new-way-to-look-at-duplication-in-fastqc-v0-11/



<hr>
## References


* https://www.ebi.ac.uk/research/enright/software/kraken
* http://hannonlab.cshl.edu/fastx_toolkit/

<br>
<br>
<hr>
github: rodriguezmDNA <br>
created: 25.07.17_jrm <br>
last revision: 25.07.17_jrm <br>
